In [1]:
#Importing libraries for sentiment analysis

import tweepy        
import pandas as pd  
import numpy as np   
import csv


#For plotting and visualization

from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
# We import our access keys:
from credential import *    # This will allow us to use the keys as variables

# API's setup:
def twitter_setup():
    """
    Utility function to setup the Twitter's API
    with our access keys provided.
    """
    # Authentication and access using keys:
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_key)

    # Return API with authentication:
    api = tweepy.API(auth,wait_on_rate_limit=True)
    return api

In [3]:
IPL2020_data = pd.DataFrame(columns = ['id', 'created_at', 'tweet_msg', 'source', 'favorite_count'])

In [6]:
extractor = twitter_setup()

xx = tweepy.Cursor(extractor.search,
                           count = 100,
                           q="#IPL2020",
                           lang="en",
                           since="2020-09-19").items(10000)

outtweets = [[tweet.id_str, tweet.created_at, tweet.text,tweet.retweet, tweet.retweet_count, tweet.source, 
              tweet.user, tweet.favorite_count, tweet.geo, tweet.coordinates, tweet.entities] for tweet in xx]
    

In [ ]:
type(outtweets)

In [ ]:
IPL2020_data = pd.DataFrame(data = outtweets, columns = ['id', 'created_at', 'tweet_msg', 'retweet', 
                                                         'retweet_count', 'source', 'user', 'favorite_count', 'geo', 
                                                        'coordinates', 'entities'])


In [ ]:
IPL2020_data.head()

In [ ]:
IPL2020_data.retweet[3]

In [ ]:
IPL2020_data.entities[3]

In [ ]:
IPL2020_data.drop(columns = ['retweet', 'user'])

In [ ]:
IPL2020_data['Length'] = IPL2020_data["tweet_msg"].str.len() 

In [ ]:
IPL2020_data

In [ ]:
IPL2020_data.isnull().sum()

### Removing the columns geo and coordinates as they are having all null values.

In [ ]:
IPL2020_data = IPL2020_data.drop(columns = ['geo', 'coordinates'])

In [ ]:
IPL2020_data

In [ ]:
IPL2020_data = IPL2020_data.drop(columns = ['id'])

In [ ]:
IPL2020_data

In [ ]:
IPL2020_data = IPL2020_data.drop(columns = ['retweet', 'user'])

In [ ]:
IPL2020_data

In [ ]:
import re

In [ ]:
type(IPL2020_data['tweet_msg'][1])

In [ ]:
IPL2020_data['tweet_msg'] = IPL2020_data['tweet_msg'].map(lambda x: str(x))

In [ ]:
type(IPL2020_data['tweet_msg'][1])

In [ ]:
TAG_CLEANING_RE = "@\S+"
# Remove @tags
IPL2020_data['tweet_msg'] = IPL2020_data['tweet_msg'].map(lambda x: re.sub(TAG_CLEANING_RE, ' ', x))

In [ ]:
IPL2020_data.tweet_msg[1]

In [ ]:
#converting the chars to lower case.
IPL2020_data['tweet_msg'] = IPL2020_data['tweet_msg'].map(lambda x: x.lower())

In [ ]:
#Removing the numbers.
IPL2020_data['tweet_msg'] = IPL2020_data['tweet_msg'].map(lambda x: re.sub(r'\d+', ' ', x))

In [ ]:
# Remove links and URLs

TEXT_CLEANING_RE = "https?:\S+|http?:\S|[^A-Za-z0-9]+"
IPL2020_data['tweet_msg'] = IPL2020_data['tweet_msg'].map(lambda x: re.sub(TEXT_CLEANING_RE, ' ', x))

In [ ]:
import string
# Remove Punctuation
IPL2020_data['tweet_msg'] = IPL2020_data['tweet_msg'].map(lambda x: x.translate(x.maketrans('', '', string.punctuation)))

In [ ]:
# Remove white spaces
IPL2020_data['tweet_msg'] = IPL2020_data['tweet_msg'].map(lambda x: x.strip())

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

In [ ]:
# Tokenize into words
IPL2020_data['tweet_msg'] = IPL2020_data['tweet_msg'].map(lambda x: word_tokenize(x))

In [ ]:
# Remove non alphabetic tokens
IPL2020_data['tweet_msg'] = IPL2020_data['tweet_msg'].map(lambda x: [word for word in x if word.isalpha()])

In [ ]:

# Filter out stop words
stop_words = set(stopwords.words('english'))
IPL2020_data['tweet_msg'] = IPL2020_data['tweet_msg'].map(lambda x: [w for w in x if not w in stop_words])

In [ ]:
# Word Lemmatization
lem = WordNetLemmatizer()
IPL2020_data['tweet_msg'] = IPL2020_data['tweet_msg'].map(lambda x: [lem.lemmatize(word,"v") for word in x])

In [ ]:

# Turn lists back to string
IPL2020_data['tweet_msg'] = IPL2020_data['tweet_msg'].map(lambda x: ' '.join(x))

In [ ]:
IPL2020_data


In [ ]:
IPL2020_data['tweet_msg'] = IPL2020_data['tweet_msg'].str.replace("rt ", '')

    



In [ ]:
IPL2020_data

In [ ]:
import nltk
#from nltk.sentiment.vader import SentimentIntensityAnalyzer
from textblob import TextBlob

# def fetch_sentiment_using_SIA(text):
#     sid = SentimentIntensityAnalyzer()
#     polarity_scores = sid.polarity_scores(text)
#     if polarity_scores['neg'] > polarity_scores['pos']:
#         return 'negative'
#     else:
#         return 'positive'

    
def fetch_sentiment_using_textblob(text):
    analysis = TextBlob(text)
    # set sentiment 
    if analysis.sentiment.polarity >= 0:
        return 'positive'
    else: 
        return 'negative'

## Performing the train test split.

In [ ]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(IPL2020_data, Y, test_size=0.3, random_state=42)
# print("TRAIN size:", len(X_train))
# print("TEST size:", len(X_test))

In [ ]:
IPL2020_data['sentiment'] = IPL2020_data['tweet_msg'].apply(lambda x: fetch_sentiment_using_textblob(x))

In [ ]:
IPL2020_data.head(10)

In [ ]:
IPL2020_data.shape

In [ ]:
IPL2020_data = IPL2020_data.drop_duplicates(subset=['tweet_msg'], keep='first')

In [ ]:
IPL2020_data.shape

In [ ]:
IPL2020_data.head(10)

In [ ]:
IPL2020_data['sentiment'].nunique()

In [ ]:
X =IPL2020_data.iloc[:,[1]]


In [ ]:
X

In [ ]:
target_variable = IPL2020_data['sentiment'].apply(lambda x: 0 if x=='negative' else 1 )

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, target_variable, test_size=0.3, random_state=42)
print("TRAIN size:", len(X_train))
print("TEST size:", len(X_test))

In [ ]:
IPL2020_data.shape


In [ ]:
type(X_train)

In [ ]:
import gensim

# WORD2VEC 
W2V_SIZE = 300
W2V_WINDOW = 7
W2V_EPOCH = 32
W2V_MIN_COUNT = 10

documents = [_text.split() for _text in X_train.tweet_msg] 
w2v_model = gensim.models.word2vec.Word2Vec(size=W2V_SIZE, 
                                            window=W2V_WINDOW, 
                                            min_count=W2V_MIN_COUNT, 
                                            workers=8)
w2v_model.build_vocab(documents)

In [ ]:
words = w2v_model.wv.vocab.keys()
vocab_size = len(words)
print("Vocab size", vocab_size)

In [ ]:
# Train Word Embeddings
w2v_model.train(documents, total_examples=len(documents), epochs=W2V_EPOCH)

In [ ]:
#Test word embeddings
w2v_model.most_similar("csk")

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Dropout
from keras.utils.np_utils import to_categorical

# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 300
# This is fixed.
EMBEDDING_DIM = 300

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train.tweet_msg)
word_index = tokenizer.word_index
vocab_size = len(word_index)
print('Found %s unique tokens.' % len(word_index))

# Convert the data to padded sequences
X_train_padded = tokenizer.texts_to_sequences(X_train.tweet_msg)
X_train_padded = pad_sequences(X_train_padded, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X_train_padded.shape)

In [ ]:
# saving
import pickle
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# Embedding matrix for the embedding layer
embedding_matrix = np.zeros((vocab_size+1, W2V_SIZE))
for word, i in tokenizer.word_index.items():
    if word in w2v_model.wv:
        embedding_matrix[i] = w2v_model.wv[word]
print(embedding_matrix.shape)

In [ ]:
embedding_matrix.shape

In [ ]:
## Embedding(8, 2, input_length=5)
## The first argument (8) is the number of distinct words in the training set. 
## The second argument (3) indicates the size of the embedding vectors.
## The input_length argument, of course, determines the size of each input sequence 
## which is the same as the max_length parameter that we used for padding.

In [ ]:
vocab_size+1, W2V_SIZE

In [ ]:
## We load this embedding matrix into an Embedding layer. 
## Note that we set trainable=False to prevent the weights from being updated during training.

In [ ]:
# This code snippet helps in optimizing the memory growth and memory utilization of the GPU.
import tensorflow as tf
config = tf.compat.v1.ConfigProto()

config.gpu_options.allow_growth = True # dynamically grow the memory used on the GPU

config.log_device_placement = True

sess = tf.compat.v1.Session(config=config)

tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))



In [ ]:
# Build Model
import keras 

model = Sequential()
model.add(Embedding(vocab_size+1, W2V_SIZE, weights=[embedding_matrix], input_length=MAX_SEQUENCE_LENGTH, trainable=False))
model.add(Dropout(0.5))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

model.summary()

model.compile(loss='binary_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])

In [ ]:

# Training 
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
callbacks = [ ReduceLROnPlateau(monitor='val_loss', patience=5, cooldown=0),
              EarlyStopping(monitor='val_acc', min_delta=1e-4, patience=5)]

BATCH_SIZE = 1024
history = model.fit(X_train_padded, y_train,
                    batch_size=512,
                    epochs=50,
                    validation_split=0.1,
                    verbose=1,
                    callbacks=callbacks)


In [ ]:
model.save_weights("Sentiment_LSTM_model.h5")
print("Saved model to disk")

In [ ]:
from keras.models import model_from_json

model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

# later...

# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

In [ ]:
# Evaluation
import matplotlib.pyplot as plt
X_test_padded = tokenizer.texts_to_sequences(X_test.tweet_msg)
X_test_padded = pad_sequences(X_test_padded, maxlen=MAX_SEQUENCE_LENGTH)
score = model.evaluate(X_test_padded, y_test, batch_size=512)
print("ACCURACY:",score[1])
print("LOSS:",score[0])

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
 
epochs = range(len(acc))
 
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
 
plt.figure()
 
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
 
plt.show()

In [ ]:
def predict(text, include_neutral=True):
    # Tokenize text
    x_test = pad_sequences(tokenizer.texts_to_sequences([text]), maxlen=MAX_SEQUENCE_LENGTH)
    # Predict
    score = model.predict([x_test])[0]
    if(score >=0.4 and score<=0.6):
        label = "Neutral"
    if(score <=0.4):
        label = "Negative"
    if(score >=0.6):
        label = "Positive"

    return {"label" : label,
        "score": float(score)}

In [ ]:
predict('CSK')

In [ ]:
predict('loss')

In [ ]:
predict('RCB')

In [ ]:
predict('rr')

In [ ]:
predict('rr')